<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/autogen/L1_Multi_Agent_Conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 1: Multi-Agent Conversation and Stand-up Comedy

Welcome to Lesson 1.

To access the `requirements.txt` file and the`utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

In [ ]:
!wget https://github.com/JSJeong-me/AutoGen/raw/main/requirements.txt

In [2]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.1/257.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.3 MB/s eta 0:00:00


In [ ]:
!echo "OPENAI_API_KEY=sk-" >> .env
!source /content/.env

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the API key using the variable name defined in the .env file
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# from google.colab import userdata
# import openai
# import os

# os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
# openai.api_key  = os.environ["OPENAI_API_KEY"]

## Setup

In [4]:
!wget https://github.com/JSJeong-me/GPT-Agent/blob/main/autogen/utils.py

--2024-11-21 20:01:38--  https://github.com/JSJeong-me/GPT-Agent/blob/main/autogen/utils.py
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘utils.py’

utils.py                [  <=>               ] 312.56K  1.01MB/s    in 0.3s    

2024-11-21 20:01:39 (1.01 MB/s) - ‘utils.py’ saved [320058]



In [5]:
import os
from dotenv import load_dotenv, find_dotenv

# these expect to find a .env file at the directory above the lesson.                                                                                                                     # the format for that file is (without the comment)                                                                                                                                       #API_KEYNAME=AStringThatIsTheLongAPIKeyFromSomeService
def load_env():
    _ = load_dotenv(find_dotenv())

def get_openai_api_key():
    load_env()
    openai_api_key = os.getenv("OPENAI_API_KEY")
    # print(openai_api_key)
    return openai_api_key

# get_openai_api_key()

In [6]:
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

## Define an AutoGen agent

In [7]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [8]:
reply = agent.generate_reply(
    messages=[{"content": "고래를 주제로 재미난 얘기 1문장으로 해줘", "role": "user"}]
)
print(reply)

고래가 바닷속에서 물속에서 '수영연습'을 좀 하다가 그만 수영을 해버렸어요, 왜냐하면 그들이 바닷속에서 가장 빠른 수영자들임을 이미 인증했기 때문이에요!


In [ ]:
reply = agent.generate_reply(
    messages=[{"content": "재미난 이야기 계속해 주세요", "role": "user"}]
)
print(reply)

## Conversation

Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [9]:
cathy = ConversableAgent(
    name="야옹님",
    system_message=
    "당신의 이름은 야옹님이고 당신은 재미난 이야기를 잘 만드는 코미디언입니다",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="멍멍님",
    system_message=
    "당신의 이름은 멍멍님이고 당신은 재미난 이야기를 잘 만드는 코미디언입니다"
    "이전 대화의 내용을 이어서 다음 재미난 이야기를 시작하세요.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

**Note**: You might get a slightly different response (set of jokes) than what is shown in the video

In [10]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="나는 야옹님이야, 우리 재미난 이야기를 이어서 나가 볼까?",
    max_turns=2,
)

멍멍님 (to 야옹님):

나는 야옹님이야, 우리 재미난 이야기를 이어서 나가 볼까?

--------------------------------------------------------------------------------
야옹님 (to 멍멍님):

네, 물론이죠! 한 번 재미난 이야기를 만들어 봅시다. 한 번 아래와 같은 시작으로 이야기를 만들어 보겠습니다.

한 번 바닷가로 휴가를 다녀온 야옹님은, 돌아오니 자신이 고양이가 아닌 사람으로 변해 있는것을 발견했습니다. 이유를 알 수 없는 상황에 야옹님이는 어떻게 대처할까요?

--------------------------------------------------------------------------------
멍멍님 (to 야옹님):

야옹님이 사람으로 변한 것을 깨닫고 처음에는 당황했습니다. 하지만 잠시 생각해보니 이게 어디서 본 듯한 상황이라고 생각했어요. 그러던 중, 바닷가에서 만난 마법사인 '모험 마법사 루루'가 이 변화의 주범임을 알게 되었죠.

마법사 루루는 장난을 좋아하는 마법사로, 야옹님에게 반가운 웃음으로 말했습니다. "안녕, 야옹님! 너를 사람으로 바꾼 건 나야. 재미있었어?"

야옹님이는 화를 내기보다 이 상황을 즐기기로 마음먹었습니다. "와, 정말 놀랍군요! 이렇게 변하게 될 줄은 몰랐어요. 하지만 괜찮아요, 이제 새로운 모험을 즐겨보자구요!"

이후 야옹님은 모험 마법사 루루와 함께 새로운 모험을 떠나기로 하고, 기존의 고양이 모습을 한 채로 마법세계를 탐험하며 웃음과 모험이 넘치는 일들을 경험했답니다. 이런 신나는 여정으로 야옹님은 더 이상 고양이의 모습을 원하지 않게 되었고, 계속해서 사람으로서의 모험을 즐기게 되었습니다. 함께 하는 여러 친구들과의 이야기와 재미난 사건들로 인생은 더욱 풍요로워졌죠. 야옹님의 새로운 모험은 계속된답니다!

-----------------------------------------------------------------------

## Print some results

You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

In [ ]:
import pprint

pprint.pprint(chat_result.chat_history)

In [ ]:
pprint.pprint(chat_result.cost)

In [ ]:
pprint.pprint(chat_result.summary)

## Get a better summary of the conversation

In [12]:
chat_result = joe.initiate_chat(
    cathy,
    message="나는 멍멍님이야, 우리 재미난 이야기를 이어서 나가 볼까?",
    max_turns=2,
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

멍멍님 (to 야옹님):

나는 멍멍님이야, 우리 재미난 이야기를 이어서 나가 볼까?

--------------------------------------------------------------------------------
야옹님 (to 멍멍님):

안녕하멍! 그럼 우리 함께 재미난 이야기를 만들어 보자! 한 번 주인님이 파티를 열기로 결심했어요. 파티 초대장을 보내기 위해 마을에 있는 마법사에게 도움을 요청했어요. 그런데 마법사는 실수로 동물들을 초대장으로 변환시켰답니다! 이런 상황에서 어떻게 해야 할까요?

--------------------------------------------------------------------------------
멍멍님 (to 야옹님):

안녕하멍! 그렇다면 우리 이야기에 마법사와 변신된 동물들이 등장했네요. 판타지한 분위기가 역동적이고 재미있겠죠! 이제 주인님은 마법사와 함께 동물들을 찾아야 할 것 같아요. 하지만 동물들은 모두 산으로 달아났고, 주인님은 마을 주변을 돌아다니면서 어디에 숨었을지 찾아야 해요. 마을 사람들이 동물들을 발견한 곳에는 초대장이 숨겨져 있을지도 모르겠어요. 주인님이 동물들을 찾을 때마다 재미난 상황들이 벌어지면서 결국에는 모두 함께 파티를 즐기는 해피 엔딩으로 마무리될 거라고 상상해봅시다! 함께 이어지는 재미난 이야기를 만들어나가요.

--------------------------------------------------------------------------------
야옹님 (to 멍멍님):

와우! 멍멍님의 아이디어는 정말 흥미진진하고 재미있는데요! 주인님은 마을 주변을 돌아다니면서 동물들을 찾기 시작했어요. 먼저 고양이로 변신된 초코가 지붕 위에서 멍멍님을 발견하고 "야옹~ 야옹~" 소리를 내며 즐거워했어요. 그리고 강아지로 변신된 보리와 토끼로 변신된 당근을 차례로 찾아냈어요. 각 동물마다 숨겨진 초대장을 발견하면서 주인님은 점점 더 흥분해졌죠.

그런데 마을 밖으로

In [ ]:
pprint.pprint(chat_result.summary)

## Chat Termination

Chat can be terminated using a termination conditions.

In [13]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [14]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey, Joe! Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, classic one! Sounds like he really had a head above the rest!

--------------------------------------------------------------------------------
cathy (to joe):

Exactly! He was a real straw-dropping performance!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's comedy gold right there! You crack me up, Cathy!

--------------------------------------------------------------------------------
cathy (to joe):

Glad I could bring a smile to your face, Joe! Laughter is the best medicine, after all!

--------------------------------------------------------------------------------
joe 

In [15]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

We were joking about the scarecrow winning an award for being outstanding in his field.

--------------------------------------------------------------------------------
cathy (to joe):

Ah, that's right! Thanks for the reminder! Have a great day, Joe!

--------------------------------------------------------------------------------
joe (to cathy):

You too, Cathy! Stay funny!

--------------------------------------------------------------------------------
cathy (to joe):

Will do! Take care!

--------------------------------------------------------------------------------
joe (to cathy):

Goodbye!

--------------------------------------------------------------------------------
cathy (to joe):

Goodbye!

--------------------------------------------------------------------------------
